In [1]:
import gc
import glob
import os
import subprocess

import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import rioxarray as rio
from rasterio.errors import RasterioIOError
from rasterio.features import geometry_mask
from rasterio.warp import Resampling, calculate_default_transform, reproject
from rioxarray.merge import merge_arrays

# Norway bathymetry

Note book to create national scale bathymetry and topo-bathymetric data for Norway.

50 m resolution bathymetric data for Norway is available from GeoNorge [here](https://kartkatalog.geonorge.no/metadata/kartverket/dybdedata-50m-grid/bbd687d0-d34f-4d95-9e60-27e330e0f76e). I have also previously created a national scale 40 m DEM by resampling the national 10 m resolution dataset. The result is here:

    shared/common/01_datasets/spatial/dtm_merged_utm33/dtm_40m/norway_kartverket_40m_dtm_utm_z33.tif

This notebook first fixes issues with the Kartverket bathymetric data, then merges the tiles to a single dataset and combines it with the 40 m  topographic dataset. The result is a single 40 m resolution topo-bathymetric dataset for Norway.

In [2]:
# Properties for output mosaic
res = 40
epsg = 25833
no_data_val = -9999
dst_dtype = "float32"  # Rasterio dtypes: https://test2.biogeo.ucdavis.edu/rasterio/_modules/rasterio/dtypes.html
bbox = (-80000, 6440000, 1122000, 7950000)  # xmin, ymin, xmax, ymax

base_dir = r"/home/jovyan/shared/common/01_datasets/spatial/norway_bathymetry_50m"
dem_path = r"/home/jovyan/shared/common/01_datasets/spatial/dtm_merged_utm33/dtm_40m/norway_kartverket_40m_dtm_utm_z33.tif"

## 1. Fix data issues

The raw Kartverket data has some issues where rounding errors in the `.xyz` data mean the y-direction of values in the file becomes reversed. This leads to `Ungridded data` errors.

In [3]:
search_path = os.path.join(base_dir, "raw/*.xyz")
flist = glob.glob(search_path)

print("The following datasets contain errors:")
for fpath in flist:
    fname = os.path.split(fpath)[1]
    try:
        rio.open_rasterio(fpath, mask_and_scale=True, cache=False)

    except RasterioIOError as e:
        if "Ungridded dataset" in str(e):
            print(f"  {fname}")
        else:
            raise

`.xyz` files are just plain text files containing lists of x, y and z coordinates. To fix the data issues, the code below reads the raw `.xyz` files and simply rounds the `x` and `y` columns to the nearest integer. It then writes them back to `.xyz` format in the `rounded` folder.

In [4]:
search_path = os.path.join(base_dir, "raw/*.xyz")
flist = glob.glob(search_path)
for fpath in flist:
    fname = os.path.split(fpath)[1]
    df = pd.read_csv(fpath, sep=" ", header=None)

    # Round the first two columns to the nearest integer
    df[0] = df[0].round().astype(int)
    df[1] = df[1].round().astype(int)

    out_csv = os.path.join(base_dir, "rounded", fname)
    df.to_csv(out_csv, sep=" ", header=False, index=False)

## 2. Merge

In [5]:
%%time

# List of DEM files to merge
search_path = os.path.join(base_dir, "rounded/*.xyz")
flist = glob.glob(search_path)

print("Opening files...")
srcs = [rio.open_rasterio(fpath, mask_and_scale=True, cache=False) for fpath in flist]

# Set original CRS for Kartverket data
srcs = [ds.rio.write_crs("epsg:25833", inplace=True) for ds in srcs]

print("Merging tiles...")
bathy_ds = merge_arrays(srcs, bounds=bbox, res=res, crs=f"epsg:{epsg}")

# Convert depths to negative
bathy_ds.data = bathy_ds.data * -1

print("Saving...")
bathy_path = os.path.join(base_dir, "merged", f"norway_{res}m_bathymetry.tif")
bathy_ds.rio.write_nodata(no_data_val, inplace=True)
bathy_ds.rio.write_crs(f"epsg:{epsg}", inplace=True)
bathy_ds.rio.to_raster(
    bathy_path,
    compress="lzw",
    BIGTIFF="YES",
    tiled=True,
    dtype=dst_dtype,
)
srcs = [src.close() for src in srcs]
bathy_ds.close()
del srcs, bathy_ds
gc.collect()

print("Done.")

Opening files...
Merging tiles...
Saving...
Done.
CPU times: user 3min 39s, sys: 16.5 s, total: 3min 55s
Wall time: 3min 56s


## 3. Combine with topographic data

In [6]:
%%time

# Read data
bathy_path = os.path.join(base_dir, "merged", f"norway_{res}m_bathymetry.tif")
bathy_ds = rio.open_rasterio(bathy_path, mask_and_scale=True, cache=False)
dem_ds = rio.open_rasterio(dem_path, mask_and_scale=True, cache=False)

# Set negative values in topo data to zero
dem_ds.data[dem_ds.data < 0] = 0

# Merge, using bathym data where available
print("Merging...")
ds = merge_arrays(
    [bathy_ds, dem_ds], method="first", bounds=bbox, res=res, crs=f"epsg:{epsg}"
)
bathy_ds.close()
dem_ds.close()
del bathy_ds, dem_ds
gc.collect()

# Save
print("Saving...")
merge_path = os.path.join(base_dir, "merged", f"norway_{res}m_topobathymetry.tif")
ds.rio.write_nodata(no_data_val, inplace=True)
ds.rio.write_crs(f"epsg:{epsg}", inplace=True)
ds.rio.to_raster(
    merge_path,
    compress="lzw",
    BIGTIFF="YES",
    tiled=True,
    dtype=dst_dtype,
)
ds.close()
del ds
gc.collect()

print("Done.")

Merging...
Saving...
Done.
CPU times: user 3min 3s, sys: 1min 15s, total: 4min 18s
Wall time: 4min 21s
